In [9]:
!pip show groq

Name: groq
Version: 0.30.0
Summary: The official Python library for the groq API
Home-page: https://github.com/groq/groq-python
Author: 
Author-email: Groq <support@groq.com>
License: Apache-2.0
Location: C:\Users\sufiyan.bt\AppData\Local\Programs\Python\Python313\Lib\site-packages
Requires: anyio, distro, httpx, pydantic, sniffio, typing-extensions
Required-by: 


In [11]:
!pip install groq langchain langchain-community unstructured python-dotenv

In [29]:
pip install sentence-transformers

Note: you may need to restart the kernel to use updated packages.


In [33]:
pip install -U langchain-huggingface

Note: you may need to restart the kernel to use updated packages.


In [34]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings

In [38]:
import os
from groq import Groq
import streamlit as st
import pickle
import time 
from dotenv import load_dotenv
from langchain_community.document_loaders import UnstructuredURLLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQAWithSourcesChain

In [ ]:
import os
os.environ["GROQ_API_KEY"] = "Api_key_here"

In [23]:
from groq import Groq

client = Groq(api_key=os.environ["GROQ_API_KEY"])

# Define your message
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Summarize fast food industry in 3 lines."}
]

# Make the call
response = client.chat.completions.create(
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    messages=messages,
    temperature=0.7,
    max_completion_tokens=256,
    stream=False,
)

# Print the response
print(response.choices[0].message.content)

The fast food industry is a multi-billion-dollar market that provides quick and affordable meals to consumers worldwide. It is dominated by major chains such as McDonald's, KFC, and Subway, as well as numerous smaller regional and local players. The industry has faced increasing pressure to offer healthier options and improve sustainability practices in response to changing consumer preferences and growing competition.


In [49]:
urls = [
    "https://www.bbc.com/news/world-asia-68422821",
    "https://www.bbc.com/news/articles/cly77v0n8e9o"
]
loader=UnstructuredURLLoader(urls=urls)
documents=loader.load()
print(f"loaded {len(documents)} documents")

loaded 2 documents


In [50]:
data = loader.load()
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)
docs = text_splitter.split_documents(data)
len(docs)

19

In [51]:
docs[3]

Document(metadata={'source': 'https://www.bbc.com/news/world-asia-68422821'}, page_content="They campaigned guerrilla-style and sprang a surprise on election day.\n\nDespite all the claims of rigging against it, his Pakistan Tehreek-e-Insaf (PTI) still emerged as the single largest party in the election. The Khan wave on election day was too strong to be reined in by routine rigging.\n\nThe establishment used 20th-Century tactics to tame a digital savvy generation - and lost.\n\nTo the military's tried and tested machinations, the voters' response was polite and defiant: thank you, but no thank you, we are not as ignorant and illiterate as you think we are. We may not be able to take you on in the streets, you have your guns, but here's our stamp on the ballot. Do what you will with it.\n\nThe seasoned agitator\n\nImran Khan didn't get a simple majority in parliament, refused to align with any other parties to form the government and decided to sit in opposition.")

In [52]:
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

vectorindex_Llama=FAISS.from_documents(docs, embedding_model)

In [53]:
file_path="vector_index.pkl"
with open(file_path,"wb") as f:
    pickle.dump(vectorindex_Llama,f)

In [54]:
if os.path.exists(file_path):
    with open(file_path , "rb") as f:
        vectorIndex=pickle.load(f)

In [56]:
def groq_retrieval_qa(query, retriever, k=3):
    docs = retriever.invoke(query)
    context = "\n\n".join([doc.page_content for doc in docs])
    sources = [doc.metadata.get("source", "N/A") for doc in docs]

    response = client.chat.completions.create(
        model="meta-llama/llama-4-scout-17b-16e-instruct",
        messages=[
            {"role": "system", "content": "Answer the user's question using the context provided. Mention sources if relevant."},
            {"role": "user", "content": f"Context:\n{context}\n\nQuestion: {query}"}
        ],
        temperature=0.7,
        max_completion_tokens=512,
        stream=False
    )

    return {
        "answer": response.choices[0].message.content,
        "sources": sources
    }

# ✅ Example usage
query = "How many years of sentence applied on Imran Khan?"
retriever = vectorIndex.as_retriever()
result = groq_retrieval_qa(query, retriever)

print("\n🔍 Answer:\n", result["answer"])
print("\n📚 Sources:\n", result["sources"])


🔍 Answer:
 According to the article, Imran Khan has been sentenced to 14 years in prison over a corruption case. This is not his only sentence, as he has received other sentences in different cases, including:

* 3 years in prison for not declaring money earned from selling gifts he had received while in office (2023)
* 14-year jail sentence over the selling of state gifts (last year, suspended months later)
* 10 years for leaking state secrets (last year, suspended months later)

However, the 14-year sentence mentioned in the article is the longest valid jail sentence he has received so far.

📚 Sources:
 ['https://www.bbc.com/news/articles/cly77v0n8e9o', 'https://www.bbc.com/news/articles/cly77v0n8e9o', 'https://www.bbc.com/news/world-asia-68422821', 'https://www.bbc.com/news/world-asia-68422821']
